# Type of examples for which punct digit predict the wrong relation vs the baseline which predicts the correct relations

In [2]:
%load_ext autoreload

In [9]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.converters.converter_i2b2 import relation_dict
output_path = '/scratch/geeticka/relation-extraction-result/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/original/test_original.txt')
punct_digit_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/punct_digit/test_punct_digit.txt')

In [10]:
relation_dict

{0: 'TrIP',
 1: 'TrWP',
 2: 'TrCP',
 3: 'TrAP',
 4: 'TrNAP',
 5: 'TeRP',
 6: 'TeCP',
 7: 'PIP',
 8: 'None'}

In [11]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [12]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [13]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [14]:
needed_linenum_and_relation = {}

In [15]:
with open(res('answers_for_dev-punct-digit.txt')) as textfile1, open(res("answers_for_dev-baseline.txt")) as textfile2, \
open(res('answers_for_dev_gold-baseline.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, punct_digit_relation = read_answers_line(x)
        _, baseline_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if baseline_relation == gold_relation and punct_digit_relation != gold_relation:
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, punct_digit_relation)

In [16]:
len(list(needed_linenum_and_relation.keys()))

963

Note that we list baseline first and then the stop words version but the prediction of the baseline version is correct (gold) whereas the stop words is incorrect. 

In [17]:
trip = 0; trwp = 0; trcp = 0; trap = 0; trnap = 0; terp = 0; tecp = 0; pip = 0; none = 0

# 0: 'TrIP',
#  1: 'TrWP',
#  2: 'TrCP',
#  3: 'TrAP',
#  4: 'TrNAP',
#  5: 'TeRP',
#  6: 'TeCP',
#  7: 'PIP',
#  8: 'None'}

In [18]:
print('We print the baseline first and then the punct digit version. Gold relation corresponds to baseline\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(punct_digit_sentences_path) as punct_digit_sentences:
    for x, y in zip(original_sentences, punct_digit_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            if r_b == 'TrIP': trip += 1
            elif r_b == 'TrWP': trwp += 1
            elif r_b == 'TrCP': trcp += 1
            elif r_b == 'TrAP': trap += 1
            elif r_b == 'TrNAP': trnap += 1
            elif r_b == 'TeRP': terp += 1
            elif r_b == 'TeCP': tecp += 1
            elif r_b == 'PIP': pip += 1
            elif r_b == 'None': none += 1
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the punct digit version. Gold relation corresponds to baseline


Predicted Relation: 	 None, TrAP 
Entities: 	 coronary artery bypass graft, coronary artery bypass graft 	 anterior myocardial infarction, anterior myocardial infarction 
Sentences: 
	2 ) Coronary artery disease , status post coronary artery bypass graft , status post anterior myocardial infarction , sick sinus syndrome , status post a VVI pacer placement . 
	 NUMBER Coronary artery disease status post coronary artery bypass graft status post anterior myocardial infarction sick sinus syndrome status post a VVI pacer placement


Predicted Relation: 	 TrAP, None 
Entities: 	 vitamin C, vitamin C 	 postoperative anemia, postoperative anemia 
Sentences: 
	Iron and vitamin C were started for postoperative anemia . 
	 Iron and vitamin C were started for postoperative anemia


Predicted Relation: 	 TeRP, TrAP 
Entities: 	 injection of agitated saline contrast, injection of agitated saline con

Predicted Relation: 	 None, TrIP 
Entities: 	 occlusion of her portal vein, occlusion of her portal vein 	 hepatic artery nearly completely occluded, hepatic artery nearly completely occluded 
Sentences: 
	CAT scan was performed on 2013-09-20 at Kindred Park View Specialty Hospital of Springfield , which revealed that she had a recurrence of the tumor with occlusion of her portal vein occluding bile ducts , hepatic artery nearly completely occluded , and much ascites and was started on diuretics . 
	 CAT scan was performed on 2013-09-20 at Kindred Park View Specialty Hospital of Springfield which revealed that she had a recurrence of the tumor with occlusion of her portal vein occluding bile ducts hepatic artery nearly completely occluded and much ascites and was started on diuretics


Predicted Relation: 	 None, PIP 
Entities: 	 occluding bile ducts, occluding bile ducts 	 hepatic artery nearly completely occluded, hepatic artery nearly completely occluded 
Sentences: 
	CAT scan was p


Predicted Relation: 	 None, PIP 
Entities: 	 increasing hepatomegaly, increasing hepatomegaly 	 increasing creatinine, increasing creatinine 
Sentences: 
	Patient returns from rehabilitation after discharge on November 11 , with new fever up to 102.0 in DSCI clinic , increasing hepatomegaly as assessed by examination in clinic , and increasing creatinine . 
	 Patient returns from rehabilitation after discharge on November NUMBER with new fever up to NUMBER in DSCI clinic increasing hepatomegaly as assessed by examination in clinic and increasing creatinine


Predicted Relation: 	 None, TrIP 
Entities: 	 rehabilitation, rehabilitation 	 new fever, new fever 
Sentences: 
	Patient returns from rehabilitation after discharge on November 11 , with new fever up to 102.0 in DSCI clinic , increasing hepatomegaly as assessed by examination in clinic , and increasing creatinine . 
	 Patient returns from rehabilitation after discharge on November NUMBER with new fever up to NUMBER in DSCI clinic

Predicted Relation: 	 TeCP, None 
Entities: 	 the prior MRI, the prior MRI 	 an underlying demyelinating process, an underlying demyelinating process 
Sentences: 
	2) Diffuse , confluent periventricular white matter hypodensity , which may relate to an underlying demyelinating process ( as suggested in the indication provided for the prior MRI ). 
	 2) Diffuse confluent periventricular white matter hypodensity which may relate to an underlying demyelinating process as suggested in the indication provided for the prior MRI


Predicted Relation: 	 None, TeRP 
Entities: 	 MRI/MRA, MRI/MRA 	 her seizures, her seizures 
Sentences: 
	MRI/MRA showed new temporal lesions , but it is unclear if these are the cause or effect of her seizures . 
	 MRI/MRA showed new temporal lesions but it is unclear if these are the cause or effect of her seizures


Predicted Relation: 	 None, TeRP 
Entities: 	 that study, that study 	 3 mm linear lesion in the left hippocampal cortex anteriorly, NUMBER mm linear

Predicted Relation: 	 None, PIP 
Entities: 	 5.5 cm poorly differentiated infiltrating ductal and intraductal carcinoma, NUMBER cm poorly differentiated infiltrating ductal and intraductal carcinoma 	 extensive lymphatic vessel invasion, extensive lymphatic vessel invasion 
Sentences: 
	The left simple mastectomy also contained 5.5 cm poorly differentiated infiltrating ductal and intraductal carcinoma with extensive lymphatic vessel invasion . 
	 The left simple mastectomy also contained NUMBER cm poorly differentiated infiltrating ductal and intraductal carcinoma with extensive lymphatic vessel invasion


Predicted Relation: 	 None, TrIP 
Entities: 	 her serum sodium, her serum sodium 	 her hyponatremia, her hyponatremia 
Sentences: 
	We proceeded to do for her hyponatremia - we gave her some fluid and her urine sodium became higher than her serum sodium ; we stopped it . 
	 We proceeded to do for her hyponatremia we gave her some fluid and her urine sodium became higher than her seru

In [19]:
print(trip, trwp, trcp, trap, trnap, terp, tecp, pip, none)

19 2 11 71 6 95 21 89 649


In [20]:
trip + trwp + trcp + trap + trnap + terp + tecp + pip + none

963